In [ ]:
import pandas as pd
import warnings, os
warnings.filterwarnings('ignore')
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Input, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Flatten, concatenate, Dense
from keras.models import Model
from tensorflow.keras.layers import Concatenate
from keras.models import Sequential
from keras.layers import Input, Concatenate, Flatten, Dense, Reshape
from keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D,MaxPool1D, Flatten, Dense, Dropout
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from keras.layers.pooling import MaxPooling2D, MaxPool1D
from sklearn.metrics import confusion_matrix
from keras.models import Model
from keras.layers import Input, Concatenate, Flatten, Dense, Reshape
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D, MaxPool1D


In [2]:
#import data
path = '/nfsmount/majid/StressDetection/Hypothesis/Morteza/backup/Final'
bio_file = 'Lasso_features_bio.csv'
land_file = 'Lasso_features_land.csv'
bio_label_file = 'data_scaled.individually_bio4.csv'
land_label_file = 'data_scaled.individually_landmark4.csv'
#import data
bio_data = pd.read_csv(os.path.join(path,bio_file)).dropna(axis=0)
land_data = pd.read_csv(os.path.join(path,land_file)).dropna(axis=0)
bio_label = pd.read_csv(os.path.join(path,bio_label_file)).dropna(axis=0)
land_label = pd.read_csv(os.path.join(path,land_label_file)).dropna(axis=0)
#extract  labels and user IDs from another file
bio_label = bio_label[['user','label']]
land_label = land_label[['user','label']]

bio_data = bio_data.join(bio_label)
land_data = land_data.join(land_label)
#Save the files for later runs
#bio_data.to_csv('fetured_bio.csv',index=False)
#land_data.to_csv('fetured_land.csv',index=False)

In [3]:
#investigate data shapes to avoid mismatchs 
bio_data.columns ,bio_data.shape, land_data.columns ,land_data.shape

(Index(['avg_TEMP', 'min_TEMP', 'energy_TEMP', 'quantile_TEMP', 'RMS_TEMP',
        'min_SCR_Height', 'std_EDA_Tonic', 'std_EDA_Phasic', 'max_EDA_Phasic',
        'min_EDA_Phasic', 'energy_EDA_Phasic', 'quantile_EDA_Phasic',
        'RMS_EDA_Phasic', 'variance_EDA_Phasic', 'avg_EDA_Clean',
        'std_EDA_Clean', 'max_EDA_Clean', 'energy_EDA_Clean', 'RMS_EDA_Clean',
        'max_HR', 'energy_HR', 'avg_EDA', 'max_EDA', 'energy_EDA',
        'quantile_EDA', 'RMS_EDA', 'variance_EDA', 'EDA_Tonic', 'EDA_Phasic',
        'TEMP', 'user', 'label'],
       dtype='object'),
 (39257, 32),
 Index(['max_X1', 'min_X1', 'energy_X1', 'variation_X1', 'energy_X5',
        'variance_X5', 'std_X3', 'energy_X3', 'max_X2', 'energy_X4',
        ...
        'min_Y53', 'avg_Y57', 'max_Y57', 'RMS_Y57', 'std_Y56', 'variation_Y67',
        'max_Y65', 'variation_Y65', 'user', 'label'],
       dtype='object', length=102),
 (39257, 102))

In [4]:
height = 10  # Height of your signal data (number of time steps)
width = 10   # Width of your signal data (number of features)
channels = 1 # Number of channels (e.g., 1 for grayscale)

![Model.png](Model.png)


In [5]:
#define the multimodal late fusion model using keras
input_bio = Input(shape=(30,))
input_land = Input(shape=(height, width, channels))
x_bio = Reshape((30, 1))(input_bio)
x_bio = Conv1D(filters=32, kernel_size=3, activation='relu')(x_bio)
x_bio = MaxPool1D(pool_size=2)(x_bio)
x_bio = Flatten()(x_bio)
x_bio = Dense(50, activation='relu')(x_bio)
x_bio = Dense(25, activation='relu')(x_bio)
output_bio = Dense(3, activation='softmax')(x_bio)
x_land = Conv2D(32, kernel_size=(3, 3), activation='relu')(input_land)
x_land = MaxPooling2D(pool_size=(2, 2))(x_land)
x_land = Flatten()(x_land)
x_land = Dense(50, activation='relu')(x_land)
x_land = Dense(25, activation='relu')(x_land)
output_land = Dense(0, activation='softmax')(x_land)

# Concatenate the two output layers
concatenated_output = Concatenate()([output_bio, output_land])
# Create the late fusion model
late_fusion_model = Model(inputs=[input_bio, input_land], outputs=concatenated_output)

# Compile the model
late_fusion_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
late_fusion_model.summary()

2023-09-21 10:34:05.583406: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-21 10:34:06.141043: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15392 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:03:00.0, compute capability: 6.0


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 30)]         0           []                               
                                                                                                  
 reshape (Reshape)              (None, 30, 1)        0           ['input_1[0][0]']                
                                                                                                  
 input_2 (InputLayer)           [(None, 10, 10, 1)]  0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 28, 32)       128         ['reshape[0][0]']                
                                                                                              

In [10]:
%%time
#leave one out cross validation using user ids from H to R
epochs = 50

for user in bio_data.user.unique():
    print(user)
    #Leave One subject Out Cross Validation train test split
    train_bio, train_land = bio_data[bio_data['user']!=user], land_data[land_data['user']!=user]
    test_bio,  test_land  = bio_data[bio_data['user']==user], land_data[land_data['user']==user]

    #split labels and features
    Xtrain_bio, Xtest_bio = train_bio.drop(columns=['user','label']), test_bio.drop(columns=['user','label'])
    ytrain_bio, ytest_bio = train_bio['label'], test_bio['label']

    
    Xtrain_land = train_land.drop(columns=['user','label'])
    #reshape training facial landmark data for conv2d
    Xtrain_land = np.reshape(Xtrain_land.iloc[:, :100].values, (Xtrain_land.shape[0], height, width, channels))

    ytrain_land = train_land['label'].values.astype(int)
    encoder = LabelEncoder()
    ytrain_land = encoder.fit_transform(ytrain_land)

    Xtest_land = test_land.drop(columns=['user','label'])
    Xtest_land = np.reshape(Xtest_land.iloc[:, :100].values, (Xtest_land.shape[0], height, width, channels))
    
    ytest_land = test_land['label']
    ytest_land = ytest_land.values.astype(int)
    encoder = LabelEncoder()
    ytest_land = encoder.fit_transform(ytest_land)

    #test station for debugging purposes
    #print('Xtrain_bio.shape',Xtrain_bio.shape, 'ytrain_bio.shape',ytrain_bio.shape, 'Xtest_bio.shape',Xtest_bio.shape, 'ytest_bio.shape',ytest_bio.shape)
    #print('Xtrain_land.shape',Xtrain_land.shape, 'ytrain_land.shape',ytrain_land.shape, 'Xtest_land.shape',Xtest_land.shape, 'ytest_land.shape',ytest_land.shape)

    y_train = tf.keras.utils.to_categorical(ytrain_bio, num_classes=3)
    y_test = tf.keras.utils.to_categorical(ytest_bio, num_classes=3)
    # Compile the model
    late_fusion_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    late_fusion_model.fit([Xtrain_bio, Xtrain_land], y_train, epochs=epochs, batch_size=32, verbose=1)

    #predict 
    y_pred = late_fusion_model.predict([Xtest_bio, Xtest_land])
    y_pred = np.argmax(y_pred, axis=1)
    y_test = y_test.argmax(axis=1)

     # Evaluate the combined predictions using the test set of model 1
    accuracy = (classification_report(y_test, y_pred, output_dict=True))

    dataframe = pd.DataFrame.from_dict(accuracy)

    with pd.ExcelWriter('2D_multi_modal_3_concatnate_batch_32.xlsx',mode="a") as writer:
        dataframe.to_excel(writer, sheet_name=user)

    # Calculate and store the confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    cm_dataframe = pd.DataFrame(cm)
    with pd.ExcelWriter('2D_multi_modal_cm_3_concatnate_batch_32.xlsx', mode="a") as writer:
        cm_dataframe.to_excel(writer, sheet_name=user) 

H
Epoch 1/50
1123/1123 [==============================] - 5s 5ms/step - loss: 0.0324 - accuracy: 0.9876
Epoch 2/50
1123/1123 [==============================] - 5s 5ms/step - loss: 0.0296 - accuracy: 0.9886
Epoch 3/50
1123/1123 [==============================] - 5s 5ms/step - loss: 0.0308 - accuracy: 0.9879
Epoch 4/50
1123/1123 [==============================] - 5s 5ms/step - loss: 0.0303 - accuracy: 0.9876
Epoch 5/50
1123/1123 [==============================] - 5s 5ms/step - loss: 0.0285 - accuracy: 0.9889
Epoch 6/50
1123/1123 [==============================] - 5s 5ms/step - loss: 0.0282 - accuracy: 0.9887
Epoch 7/50
1123/1123 [==============================] - 5s 5ms/step - loss: 0.0275 - accuracy: 0.9891
Epoch 8/50
1123/1123 [==============================] - 5s 5ms/step - loss: 0.0298 - accuracy: 0.9884
Epoch 9/50
1123/1123 [==============================] - 5s 5ms/step - loss: 0.0264 - accuracy: 0.9896
Epoch 10/50
1123/1123 [==============================] - 5s 5ms/step - loss: 0.0

In [8]:
%%time
user = 'I'
epochs = 50


print(user)
#Leave One subject Out Cross Validation train test split
train_bio, train_land = bio_data[bio_data['user']!=user], land_data[land_data['user']!=user]
test_bio,  test_land  = bio_data[bio_data['user']==user], land_data[land_data['user']==user]

#split labels and features
Xtrain_bio, Xtest_bio = train_bio.drop(columns=['user','label']), test_bio.drop(columns=['user','label'])
ytrain_bio, ytest_bio = train_bio['label'], test_bio['label']


Xtrain_land = train_land.drop(columns=['user','label'])
#reshape training facial landmark data for conv2d
Xtrain_land = np.reshape(Xtrain_land.iloc[:, :100].values, (Xtrain_land.shape[0], height, width, channels))

ytrain_land = train_land['label'].values.astype(int)
encoder = LabelEncoder()
ytrain_land = encoder.fit_transform(ytrain_land)

Xtest_land = test_land.drop(columns=['user','label'])
Xtest_land = np.reshape(Xtest_land.iloc[:, :100].values, (Xtest_land.shape[0], height, width, channels))

ytest_land = test_land['label']
ytest_land = ytest_land.values.astype(int)
encoder = LabelEncoder()
ytest_land = encoder.fit_transform(ytest_land)

#test station for debugging purposes
#print('Xtrain_bio.shape',Xtrain_bio.shape, 'ytrain_bio.shape',ytrain_bio.shape, 'Xtest_bio.shape',Xtest_bio.shape, 'ytest_bio.shape',ytest_bio.shape)
#print('Xtrain_land.shape',Xtrain_land.shape, 'ytrain_land.shape',ytrain_land.shape, 'Xtest_land.shape',Xtest_land.shape, 'ytest_land.shape',ytest_land.shape)

y_train = tf.keras.utils.to_categorical(ytrain_bio, num_classes=3)
y_test = tf.keras.utils.to_categorical(ytest_bio, num_classes=3)
late_fusion_model.fit([Xtrain_bio, Xtrain_land], y_train, epochs=epochs, batch_size=32, verbose=1)

  

I
Epoch 1/50
1104/1104 [==============================] - 5s 5ms/step - loss: 0.0164 - accuracy: 0.9937
Epoch 2/50
1104/1104 [==============================] - 5s 5ms/step - loss: 0.0137 - accuracy: 0.9947
Epoch 3/50
1104/1104 [==============================] - 5s 5ms/step - loss: 0.0149 - accuracy: 0.9947
Epoch 4/50
1104/1104 [==============================] - 5s 5ms/step - loss: 0.0128 - accuracy: 0.9952
Epoch 5/50
1104/1104 [==============================] - 5s 5ms/step - loss: 0.0137 - accuracy: 0.9952
Epoch 6/50
1104/1104 [==============================] - 5s 5ms/step - loss: 0.0127 - accuracy: 0.9953
Epoch 7/50
1104/1104 [==============================] - 5s 5ms/step - loss: 0.0124 - accuracy: 0.9957
Epoch 8/50
1104/1104 [==============================] - 5s 5ms/step - loss: 0.0120 - accuracy: 0.9955
Epoch 9/50
1104/1104 [==============================] - 5s 5ms/step - loss: 0.0125 - accuracy: 0.9953
Epoch 10/50
1104/1104 [==============================] - 5s 5ms/step - loss: 0.0

In [9]:
%%time
#predict 
y_pred = late_fusion_model.predict([Xtest_bio, Xtest_land])
y_pred = np.argmax(y_pred, axis=1)
y_test = y_test.argmax(axis=1)

  # Evaluate the combined predictions using the test set of model 1
accuracy = (classification_report(y_test, y_pred, output_dict=True))

dataframe = pd.DataFrame.from_dict(accuracy)


CPU times: user 519 ms, sys: 66.6 ms, total: 586 ms
Wall time: 456 ms


: 

In [ ]:

with pd.ExcelWriter('2D_multi_modal_3_concatnate_batch_32.xlsx',mode="a") as writer:
    dataframe.to_excel(writer, sheet_name=user)

# Calculate and store the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_dataframe = pd.DataFrame(cm)
with pd.ExcelWriter('2D_multi_modal_cm_3_concatnate_batch_32.xlsx', mode="a") as writer:
    cm_dataframe.to_excel(writer, sheet_name=user) 